# Single arm motion planning

This notebook shows how to use the SingleArmOmplPlanner with Drake to plan a collision-free path between two joint configurations.

In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_meshcat_to_builder, finish_build
from cloth_tools.drake.scenes import add_ur5e_and_table_to_builder
from cloth_tools.drake.visualization import publish_joint_path
from cloth_tools.ompl.single_arm_planner import SingleArmOmplPlanner

## 1. Setting up Drake for collision checking 

### 1.1 Start building the Drake diagram

Building Drake diagram can be quite complex. 
To simplify this Drake has a helper class called `RobotDiagramBuilder` that helps linking all the Drake systems.

Using this class and its attirbutes we can easily add URDFs and weld them together.
As far as I know, a weld is exactly the same as a fixed joint in URDF.

In [ ]:
robot_diagram_builder = RobotDiagramBuilder()  # time_step=0.001 even when I set timestep I get the mimic joint warning
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_index, gripper_index = add_ur5e_and_table_to_builder(robot_diagram_builder)

### 1.2 Finalize the diagram

In [ ]:
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

### 1.3 Creating the collision checker

In [ ]:
collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index, gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

In [ ]:
start_joints = np.deg2rad([0, -90, -90, -90, 90, 0])
collision_checker.CheckConfigCollisionFree(start_joints)

In [ ]:
# To visualize the start pose in meshcat
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

In [ ]:
goal_joints = np.deg2rad([180, -135, 95, -50, -90, -90])
collision_checker.CheckConfigCollisionFree(goal_joints)

## 2. Integration with OMPL

For integration with OMPL, we provide a `SingleArmOmplPlanner`, which handles conversion to OMPL data types and creates OMPL objects. 

The only "external" thing this class requires, a function tells it whether a given joint configuration is valid or not.

In this case, the `CheckConfigCollisionFree` method of the `SceneGraphCollisionChecker` we made can be used for this purpose.

In [ ]:
planner = SingleArmOmplPlanner(is_state_valid_fn=collision_checker.CheckConfigCollisionFree)

# You can choose the planner OMPL uses like so:
# import ompl.geometric as og
# simple_setup = planner._simple_setup
# simple_setup.setPlanner(og.LBKPIECE1(simple_setup.getSpaceInformation()))

In [ ]:
path = planner.plan_to_joint_configuration(start_joints, goal_joints)

## 3. Visualizing the path in Drake

Run the cell below and look at Meshcat in your browser to see your robot moving.

In [ ]:
publish_joint_path(path, 5.0, meshcat, diagram, context, arm_index)